In [120]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from io import StringIO
from datetime import datetime, timedelta
# Add this at the top of your notebook
import warnings
warnings.filterwarnings('ignore')

## Scrape one day

In [101]:
day=7;month=11;year=2025
url = f'https://uec.hse.ie/uec/TGAR.php?EDDATE={day}%2F{month}%2F{year}'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table')
df = pd.read_html(str(table), skiprows=[0])[0]
#df.columns = df.columns.droplevel(0)


/var/folders/8p/bnwtr4b93fn3_th1x9ysmwy00000gn/T/ipykernel_12825/348012980.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), skiprows=[0])[0]


In [ ]:
regions=['HSE Dublin & North East','HSE Dublin & Midlands','HSE Dublin & South East','HSE South West','HSE Mid West','HSE West & North West']
df_cp = df.copy()# Create a copy to manipulate

# Replace region names with NaN
df_cp = df_cp.replace(regions, np.nan)
df_cp = df_cp.dropna(axis=1, how='all')  # Remove all-NaN columns
df_cp = df_cp.dropna(axis=0, how='all')  # Remove all-NaN rows

# cut first 8 columns, there are just repeats
col_to_cut=8
df_cp = df_cp.iloc[:, col_to_cut:]
df_cp = df_cp.drop('Unnamed: 9_level_0', axis=1)

# Rename hospital column
df_cp = df_cp.rename(columns={'Unnamed: 8_level_0': 'Hospital'})
# move 2nd multi index to first row
# get second header
second_level = df_cp.columns.get_level_values(1).tolist()
# drop second header
df_cp.columns = df_cp.columns.droplevel(1)
# Insert as first row
df_cp.loc[0] = second_level
# set date
df_cp.insert(0, 'Date', f'{day:02d}/{month:02d}/{year}')#inplace by default
# print it out
# df_cp

/var/folders/8p/bnwtr4b93fn3_th1x9ysmwy00000gn/T/ipykernel_12825/2207899642.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cp = df_cp.replace(regions, np.nan)
/var/folders/8p/bnwtr4b93fn3_th1x9ysmwy00000gn/T/ipykernel_12825/2207899642.py:12: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_cp = df_cp.drop('Unnamed: 9_level_0', axis=1)


,Date,Hospital,ED Trolleys,Ward Trolleys,Total,Surge Capacity in Use (Full report @14:00),Delayed Transfers of Care (As of Midnight),No of Total Waiting >24hrs,No of >75+yrs Waiting >24hrs
2,07/11/2025,Cavan General Hospital,6,0,6,13,NaN,NaN,NaN
4,07/11/2025,Connolly Hospital,0,0,0,0,NaN,NaN,NaN
6,07/11/2025,Louth County Hospital,NaN,NaN,NaN,NaN,0,NaN,NaN
8,07/11/2025,Mater Misericordiae University Hospital,16,0,16,12,NaN,NaN,NaN
10,07/11/2025,National Orthopaedic Hospital Cappagh,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,07/11/2025,Our Lady of Lourdes Hospital,0,0,0,22,NaN,NaN,NaN
14,07/11/2025,Our Lady's Hospital Navan,0,0,0,0,NaN,NaN,NaN
15,07/11/2025,HSE Dublin and North East Total,34,0,34,75,0,0,0
18,07/11/2025,CHI at Tallaght,0,0,0,0,NaN,NaN,NaN
20,07/11/2025,CHI at Temple Street,0,0,0,0,NaN,NaN,NaN


In [109]:
hospital_to_region_dict = {
    # HSE Dublin & North East
    "Beaumont Hospital":                        "HSE Dublin & North East",
    "Cavan General Hospital":                   "HSE Dublin & North East",
    "Connolly Hospital":                        "HSE Dublin & North East",
    "Louth County Hospital":                    "HSE Dublin & North East",
    "Mater Misericordiae University Hospital":  "HSE Dublin & North East",
    "National Orthopaedic Hospital Cappagh":    "HSE Dublin & North East",
    "Our Lady of Lourdes Hospital":             "HSE Dublin & North East",
    "Our Lady's Hospital Navan":                "HSE Dublin & North East",

    # HSE Dublin & Midlands
    "CHI at Crumlin":                           "HSE Dublin & Midlands",
    "CHI at Tallaght":                          "HSE Dublin & Midlands",
    "CHI at Temple Street":                     "HSE Dublin & Midlands",
    "MRH Mullingar":                            "HSE Dublin & Midlands",
    "MRH Portlaoise":                           "HSE Dublin & Midlands",
    "MRH Tullamore":                            "HSE Dublin & Midlands",
    "Naas General Hospital":                    "HSE Dublin & Midlands",
    "St. James's Hospital":                     "HSE Dublin & Midlands",
    "St. Luke's Radiation Oncology Network":    "HSE Dublin & Midlands",
    "Tallaght University Hospital":             "HSE Dublin & Midlands",

    # HSE Dublin & South East
    "National Rehabilitation Hospital":         "HSE Dublin & South East",
    "St. Columcille's Hospital":                "HSE Dublin & South East",
    "St. Luke's General Hospital Kilkenny":     "HSE Dublin & South East",
    "St. Michael's Hospital":                   "HSE Dublin & South East",
    "St. Vincent's University Hospital":        "HSE Dublin & South East",
    "Tipperary University Hospital":            "HSE Dublin & South East",
    "UH Waterford":                             "HSE Dublin & South East",
    "Wexford General Hospital":                 "HSE Dublin & South East",

    # HSE South West
    "Bantry General Hospital":                      "HSE South West",
    "Cork University Hospital":                     "HSE South West",
    "Mallow General Hospital":                      "HSE South West",
    "Mercy University Hospital":                    "HSE South West",
    "South Infirmary Victoria University Hospital": "HSE South West",
    "UH Kerry":                                     "HSE South West",

    # HSE Mid West
    "Ennis Hospital":                               "HSE Mid West",
    "Nenagh Hospital":                              "HSE Mid West",
    "St. John's Hospital Limerick":                 "HSE Mid West",
    "UH Limerick":                                  "HSE Mid West",

    # HSE West & North West
    "Galway University Hospital":                   "HSE West & North West",
    "Letterkenny University Hospital":              "HSE West & North West",
    "Mayo University Hospital":                     "HSE West & North West",
    "Portiuncula University Hospital":              "HSE West & North West",
    "Roscommon University Hospital":                "HSE West & North West",
    "Sligo University Hospital":                    "HSE West & North West"
}


In [110]:
df_cp['Region'] = df_cp['Hospital'].map(hospital_to_region_dict)
df_cp

,Date,Hospital,ED Trolleys,Ward Trolleys,Total,Surge Capacity in Use (Full report @14:00),Delayed Transfers of Care (As of Midnight),No of Total Waiting >24hrs,No of >75+yrs Waiting >24hrs,Region
2,07/11/2025,Cavan General Hospital,6,0,6,13,NaN,NaN,NaN,HSE Dublin & North East
4,07/11/2025,Connolly Hospital,0,0,0,0,NaN,NaN,NaN,HSE Dublin & North East
6,07/11/2025,Louth County Hospital,NaN,NaN,NaN,NaN,0,NaN,NaN,HSE Dublin & North East
8,07/11/2025,Mater Misericordiae University Hospital,16,0,16,12,NaN,NaN,NaN,HSE Dublin & North East
10,07/11/2025,National Orthopaedic Hospital Cappagh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HSE Dublin & North East
12,07/11/2025,Our Lady of Lourdes Hospital,0,0,0,22,NaN,NaN,NaN,HSE Dublin & North East
14,07/11/2025,Our Lady's Hospital Navan,0,0,0,0,NaN,NaN,NaN,HSE Dublin & North East
15,07/11/2025,HSE Dublin and North East Total,34,0,34,75,0,0,0,NaN
18,07/11/2025,CHI at Tallaght,0,0,0,0,NaN,NaN,NaN,HSE Dublin & Midlands
20,07/11/2025,CHI at Temple Street,0,0,0,0,NaN,NaN,NaN,HSE Dublin & Midlands


## Scrape 3 years into 6 .csv files

In [140]:
# set date
#day=7;month=11;year=2025
# scrape the html
def scrape_date(day, month, year):
  #print(f'Scraping date: {day:02d}/{month:02d}/{year}')
  url = f'https://uec.hse.ie/uec/TGAR.php?EDDATE={day}%2F{month}%2F{year}'
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  table = soup.find('table')
  df = pd.read_html(StringIO(str(table)), skiprows=[0])[0]
  df_cp = df.copy()# Create a copy to manipulate

  #df.columns = df.columns.droplevel(0)
  regions=['HSE Dublin & North East','HSE Dublin & Midlands','HSE Dublin & South East','HSE South West','HSE Mid West','HSE West & North West']

  # Replace region names with NaN
  df_cp = df_cp.replace(regions, np.nan).infer_objects(copy=False)
  df_cp = df_cp.dropna(axis=1, how='all')  # Remove all-NaN columns
  df_cp = df_cp.dropna(axis=0, how='all')  # Remove all-NaN rows

  # cut first 8 columns, there are just repeats
  col_to_cut=8
  df_cp = df_cp.iloc[:, col_to_cut:]
  df_cp = df_cp.drop('Unnamed: 9_level_0', axis=1)

  # Rename hospital column
  df_cp = df_cp.rename(columns={'Unnamed: 8_level_0': 'Hospital'})
  # move 2nd multi index to first row
  # get second header
  second_level = df_cp.columns.get_level_values(1).tolist()
  # drop second header
  df_cp.columns = df_cp.columns.droplevel(1)
  # Insert as first row
  df_cp.loc[0] = second_level
  # set date
  df_cp.insert(0, 'Date', f'{day:02d}/{month:02d}/{year}')#inplace by default
  # print it out
  # df_cp
  df_cp['Region'] = df_cp['Hospital'].map(hospital_to_region_dict)
  return df_cp 

In [141]:
# Get all dates for past 3 years
end_date    = datetime(2025, 11, 15)
start_date  = datetime(2023, 1, 1)
dates       = pd.date_range(start=start_date, end=end_date, freq='D')

### 6 month chunks

### start with 2023

In [142]:
start_date  = datetime(2023, 1, 1);end_date    = datetime(2023, 6, 1)
dates       = pd.date_range(start=start_date, end=end_date, freq='D')
pd.concat([scrape_date(date.day, date.month, date.year) for date in dates], ignore_index=True).to_csv('uec_data_2023p1.csv', index=False)

KeyboardInterrupt: 

In [ ]:
start_date  = datetime(2023, 6, 2);end_date    = datetime(2023, 12, 31)
dates       = pd.date_range(start=start_date, end=end_date, freq='D')
pd.concat([scrape_date(date.day, date.month, date.year) for date in dates], ignore_index=True).to_csv('uec_data_2023p2.csv', index=False)

### 2024

In [ ]:
start_date  = datetime(2024, 1, 1);end_date    = datetime(2024, 6, 1)
dates       = pd.date_range(start=start_date, end=end_date, freq='D')
pd.concat([scrape_date(date.day, date.month, date.year) for date in dates], ignore_index=True).to_csv('uec_data_2024p1.csv', index=False)

In [ ]:
start_date  = datetime(2024, 6, 2);end_date    = datetime(2024, 12, 31)
dates       = pd.date_range(start=start_date, end=end_date, freq='D')
pd.concat([scrape_date(date.day, date.month, date.year) for date in dates], ignore_index=True).to_csv('uec_data_2024p2.csv', index=False)

### 2025

In [ ]:
start_date  = datetime(2025, 1, 1);end_date    = datetime(2025, 6, 1)
dates       = pd.date_range(start=start_date, end=end_date, freq='D')
pd.concat([scrape_date(date.day, date.month, date.year) for date in dates], ignore_index=True).to_csv('uec_data_2025p1.csv', index=False)

In [ ]:
start_date  = datetime(2025, 6, 2);end_date= datetime(2025,11, 15)
dates       = pd.date_range(start=start_date, end=end_date, freq='D')
pd.concat([scrape_date(date.day, date.month, date.year) for date in dates], ignore_index=True).to_csv('uec_data_2025p2.csv', index=False)

## Concat 3 years/6 files to one

In [145]:
pd.concat([pd.read_csv(f'data/uec_data_2023p1.csv'),
           pd.read_csv(f'data/uec_data_2023p2.csv'),
           pd.read_csv(f'data/uec_data_2024p1.csv'),
           pd.read_csv(f'data/uec_data_2024p2.csv'),
           pd.read_csv(f'data/uec_data_2025p1.csv'),
           pd.read_csv(f'data/uec_data_2025p2.csv'),], 
           ignore_index=True).to_csv('data/uec_data_2023_2025_full.csv', index=False)